In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [97]:
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource)

In [107]:
partition <- caret::createDataPartition(y=df.training$sale_price_log, p=.6, list=F, times=1)

trainset <- df.training[partition,] %>% select(-SalePrice)
testset <- df.training[-partition,] %>% select(-sale_price_log)

stopifnot(setdiff(trainset %>% colnames, testset %>% colnames) == 'sale_price_log')
stopifnot(setdiff(testset %>% colnames, trainset %>% colnames) == 'SalePrice')

tran.res <- kaggle.house$trans$doItAll(trainset, testset %>% select(-SalePrice))
trainset.ready <- tran.res$trainset
testset.ready <- tran.res$testset

# Should be only numeric columns
stopifnot(0 == trainset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)
stopifnot(0 == testset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

stopifnot(setdiff(trainset.ready %>% colnames, testset.ready %>% colnames) == 'sale_price_log')
stopifnot(setdiff(testset.ready %>% colnames, trainset.ready %>% colnames) == '')

# sanity check
#testset %>% group_by(BldgType) %>% summarise(median(log(SalePrice)))
#cbind(testset.ready %>% select(BldgType.new), testset %>% select(BldgType)) %>% 
#    group_by(BldgType) %>% summarise(min(BldgType.new), max(BldgType.new))

model.lm <- lm(sale_price_log ~ ., data=trainset.ready)

suppressWarnings({
    y_predicted <- predict(model.lm, testset.ready) %>% as.vector
})
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)

paste0("Testset RMSE of sale_price_log: ", rmse(y_predicted, y_actual))

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)

paste0("Testset SalePrice mean prediction error in %: ", mean(100 * abs(sp_actual - sp_predicted) / sp_actual))

glance(model.lm)
tidy(model.lm) %>% arrange(p.value) %>% filter(p.value < 0.1)

[1] "Testset RMSE of sale_price_log: 0.139268551268354"

[1] "Testset SalePrice mean prediction error in %: 10.053643037849"

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.9032026,0.8940183,0.1257676,98.34228,0,77,614.8402,-1073.68,-701.024,12.66981,801


term,estimate,std.error,statistic,p.value
Neighborhood.new,2.831238e-01,2.735925e-02,10.348373,1.215210e-23
OverallQual,5.066913e-02,6.332933e-03,8.000895,4.323464e-15
has_pool,-3.136063e+00,4.492431e-01,-6.980770,6.163344e-12
PoolArea,5.044335e-03,7.498415e-04,6.727201,3.291033e-11
X1stFlrSF,1.983647e-04,3.000340e-05,6.611407,6.948901e-11
OverallCond,3.338949e-02,5.734437e-03,5.822628,8.381679e-09
X2ndFlrSF,1.267307e-04,2.500392e-05,5.068432,4.984195e-07
BsmtFullBath,5.081346e-02,1.273792e-02,3.989150,7.238399e-05
HeatingQC.new,1.292361e-01,3.384704e-02,3.818240,1.447829e-04
Condition1.new,2.146989e-01,5.794800e-02,3.705026,2.258917e-04


In [77]:
source('main.R')

suppressWarnings({
    N = 50

    runs <- c(1:N) %>% map(function (i) { 
        sample <- df.training %>% sample_n(round(0.75 * nrow(df.training)))
        sample.partition.index <- caret::createDataPartition(y=sample$sale_price_log, p=.6, list=F, times=1)

        trainset <- sample[sample.partition.index,] %>% select(-SalePrice)
        testset <- sample[-sample.partition.index,] %>% select(-sale_price_log)

        tranform.results <- kaggle.house$trans$doItAll(trainset, testset)

        trainset <- tranform.results$trainset
        testset <- tranform.results$testset

        model <- lm(sale_price_log ~ ., data=trainset)
        y_predicted <- predict(model, testset) %>% as.vector
        y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)

        list(model=model, y_test_predicted=y_predicted, y_test_actual=y_actual)
    })    
})

In [96]:
rmse.sample <- 
    runs %>%
    map(function(item) { rmse(item$y_test_actual, item$y_test_predicted) }) %>% 
    unlist %>% as.vector

SalePrice.predictionError.sample <- 
    runs %>%
    map(function(item) { 
            actual <- item$y_test_actual
            predicted <- item$y_test_predicted
            mean(100 * abs(exp(actual) - exp(predicted)) / exp(actual)) 
    }) %>% 
    unlist %>% as.vector

r2.sample <- 
    runs %>% 
    map(function(item) { glance(item$model)[['r.squared']] }) %>% 
    unlist %>% as.vector

coeff.sample <- 
    runs %>% 
    map(function(item) { tidy(item$model) %>% select(term, estimate, p.value) }) %>%
    bind_rows %>% arrange(term)

summary(rmse.sample)
summary(SalePrice.predictionError.sample)
summary(r2.sample)

coeff.sample %>% filter(term == 'OverallQual') %>% select(estimate) %>% summary
coeff.sample %>% filter(term == 'Neighborhood.new') %>% select(estimate) %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1165  0.1350  0.1522  0.1584  0.1747  0.2807 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  8.294   9.735  10.362  10.976  11.184  29.313 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.9063  0.9138  0.9201  0.9220  0.9295  0.9437 

    estimate      
 Min.   :0.03311  
 1st Qu.:0.04880  
 Median :0.05322  
 Mean   :0.05319  
 3rd Qu.:0.05732  
 Max.   :0.07169  

    estimate      
 Min.   :0.09182  
 1st Qu.:0.13161  
 Median :0.19841  
 Mean   :0.18593  
 3rd Qu.:0.22514  
 Max.   :0.29069  

In [36]:
coeff.sample <- fits %>% map(function(item) { tidy(item$model) }) %>% bind_rows
coeff.sample %>% 
    arrange(term, estimate) %>% 
    group_by(term) %>% 
    mutate(n=n()) %>% 
    filter(p.value < 0.00000001, n == N) %>%
    distinct(term) %>%
    `[[`(1) %>% paste(collapse = '+')

[1] "BsmtFinSF1+Condition2.new+has_pool+MSZoning.new+Neighborhood.new+OverallCond+OverallQual+PoolArea+X1stFlrSF+X2ndFlrSF"

In [160]:
df.tr <- 
    trainset.ready %>% 
    mutate(
        GrLivArea.log = log(GrLivArea),
        X1stFlrSF.log = log(X1stFlrSF),
        X2ndFlrSF.log = log(X2ndFlrSF + 1),
        LotArea.log = log(LotArea)
    )

df.te <- 
    testset.ready %>% 
    mutate(
        GrLivArea.log = log(GrLivArea),
        X1stFlrSF.log = log(X1stFlrSF),
        X2ndFlrSF.log = log(X2ndFlrSF + 1),
        LotArea.log = log(LotArea)
    )

model.lm <- lm(
    sale_price_log ~ OverallQual+Neighborhood.new+GrLivArea.log+X1stFlrSF.log+BsmtFullBath+
                     LotArea.log+has_central_air+YearRemodAdd+GarageCars, 
    data=df.tr)


glance(model.lm)
tidy(model.lm)

y_predicted <- predict(model.lm, df.te) %>% as.vector
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)
rmse(y_predicted, y_actual)

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)
mean(100 * abs(sp_actual - sp_predicted) / sp_actual)

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.8668367,0.8651745,0.1483862,521.4895,1.039939e-308,10,362.4922,-702.9843,-652.4458,15.87532,721


term,estimate,std.error,statistic,p.value
(Intercept),0.51335179,0.6846550006,0.7497963,4.536220e-01
OverallQual,0.08052613,0.0069156870,11.6439812,7.804972e-29
Neighborhood.new,0.27587930,0.0272502951,10.1239013,1.292317e-22
GrLivArea.log,0.31553726,0.0235967954,13.3720388,1.365706e-36
X1stFlrSF.log,0.09243856,0.0232050584,3.9835522,7.478674e-05
BsmtFullBath,0.06390370,0.0113162861,5.6470563,2.348617e-08
LotArea.log,0.08222360,0.0128285174,6.4094389,2.638188e-10
has_central_air,0.16017446,0.0221639189,7.2268112,1.262187e-12
YearRemodAdd,0.00187511,0.0003446257,5.4410047,7.264625e-08
GarageCars,0.06376398,0.0100036715,6.3740579,3.284150e-10


[1] 0.141228

[1] 10.5194